 1. 这个需要你先用历史波动率c2c模型分别计算出不同到期日的atm的波动率，
 2. 之后再根据wing模型，具体参数你可以做一个参数组，分为微笑（曲率：小中大），右偏（曲率：小中大）：小中大，左偏（曲率：小中大），计算出一个波动率曲面，之后把上面那两个表自动生成出来。

- 另外，行权价距离atm值15%的位置的波动率是atm波动率的30%，6%的位置波动率是atm波动率的16%，这样就能生成5个点，你用这5点拟合一下图形，把剩下的生成出来就行了

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
%matplotlib inline

from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
def phi(A):
    """
    平滑函数： 
    $$ \varphi(\theta) = \frac{\eta}{\theta^\gamma(1+\theta)^{1-\gamma}} $$
    """
    return A[2] / (A[0]**A[3]*(1+A[0])**(1-A[3]))

def model(A, x):
    """
    SSVI 函数
    $$ w(k, \theta_t) = \frac{\theta_t}{2} \left \{ 1 + \rho \varphi(\theta_t)k + \sqrt{(\varphi(\theta_t)k+\rho)^2 + (1-\rho^2)} \right \} $$
    """
    return A[0]/2 * (1 + A[1]*phi(A)*x + ((phi(A)*x + A[1])**2 + 1 - A[1]**2)**0.5)

def fun_penalty(A, x, y,paramater_pre):
    """
    优化目标函数
    """
    least_squar = (model(A, x) - y)**2  ## 最小二乘
    penalty_eta =  1*(A[2] * (1+np.abs(A[1])) - 2)**2  ## 无蝶式套利约束条件
    c = model(A, x) - model(paramater_pre, x)   ## 无 time spread 套利
    penalty_maturity = np.sum(c[c<0]) **2
    return  least_squar + penalty_maturity  #+ penalty_eta

def fit_svi(strike, variance):
    """
    拟合SVI，得到对应系数
    
    参数：
    strike: 1d array $ln(k/s)$ 对数行权价
    variance : 2d array 总体方差， $\sigma^2*t$
    """
    paramater = np.random.rand(variance.shape[0],4)
    paramater_pre = np.ones(4) / 10
    for i in range(4):
        res = least_squares(fun_penalty, paramater_pre, args=(strike, variance[i], paramater_pre), 
                            bounds=((-np.inf, -np.inf, -np.inf, 0),(np.inf, np.inf, np.inf, 0.5)))
        paramater[i] = res.x
        paramater_pre = res.x
    return paramater

In [3]:
def get_stock_data(code_dir,period, start):
    pre_start=w.tdaysoffset(-np.max(period), start, "Period=D").Data[0][0]
    data = w.wsd(code_dir, "pre_close,high,low,close,lastradeday_s", pre_start, start)
    df = pd.DataFrame(data.Data, index=data.Fields).T
    df['DailyReturn'] = df['CLOSE'] / df['PRE_CLOSE']    
    df['LASTRADEDAY_S'] = pd.to_datetime(df['LASTRADEDAY_S'])
    df.set_index('LASTRADEDAY_S', inplace=True)
    return df   

In [4]:
def calculate_vol(data, start,period, method='CLC'):
    ## ewma
    if method == 'EWMA':
        his_vol = [((np.log(data['DailyReturn'].astype('float32'))**2).ewm(span=day).mean())[-1]**0.5*252**0.5 for day in period ]
    
    ## clc
    elif method == 'CLC': 
        his_vol = np.array([(np.log(data['DailyReturn'].astype('float32')).rolling(window=day).std())[-1]*252**0.5  for day in period])
    
    ## Parkinson
    elif method == 'PARKINSON': 
        his_vol = [(data['HIGH']/data['LOW']**2/(4*np.log(2))).rolling(window=day).mean()[-1]*252**0.5 for day in period]

    return his_vol

In [5]:
def output_iv(file_dir, start=None,  method='CLC'):
    start = start if start else str(pd.datetime.today().date())
    code_dir_info = os.path.basename(file_dir).split('.')
    if len(code_dir_info) == 3:    
        code_dir = '.'.join(code_dir_info[:-1]) 
    elif code_dir_info[0][0] in ['0', '1', '2', '3']:
        code_dir = code_dir_info[0] + '.SZ'
    else:
        code_dir = code_dir_info[0] + '.SH'
    ## basic info
    basic_data = pd.read_excel(file_dir)
    columns = basic_data.columns
    period = (pd.to_datetime(basic_data.loc[basic_data['C/P']=='C'].index.astype('str')) - pd.datetime.today()).days.values  ## maturity
    predict_k = basic_data.columns[2:].astype('float16').values  ## strike

    ## 得到历史波动率
    df = get_stock_data(code_dir, period, start)
    ATM = df['CLOSE'][-1]
    his_vol = calculate_vol(df, start, period, method)

    ## 构造波动率数据，这里可以修改参数
    Vol_data = pd.DataFrame([his_vol*(1+0.3), his_vol*(1+0.16), his_vol, his_vol*(1+0.16), his_vol*(1+0.3)],
                 index=[ATM*(1-0.15), ATM*(1-0.06), ATM, ATM*(1+0.06), ATM*(1+0.15) ], columns=period).T

    ## 拟合波动率 得到系数
    strike = np.log(Vol_data.columns.values / ATM)
    marturity = Vol_data.index.values.reshape((-1,1))
    implied_volatility = Vol_data.values
    variance = implied_volatility**2 * marturity

    paramater = fit_svi(strike, variance)

    ## 得到结果
    result_data = basic_data.copy()
    result = [(model(paramater[i], np.log(predict_k/ATM)) / marturity[i])**0.5 *100  for i in range(paramater.shape[0])]
    result_data.iloc[::2, 2:] = result
    result_data.iloc[1::2,2:] = result

    ## output 
    result_data.to_excel(os.path.join('./output',os.path.basename(file_dir)))
    
    return result_data

In [6]:
def transfer_folder(folder, start=None, method='CLC'):
    for file_dir in os.listdir(folder):
        try:
            output = output_iv(os.path.join(folder,file_dir))
        except:
            print('Error:', file_dir)

In [7]:
transfer_folder('./data')